### 1. Data Ingestion

In [4]:
import pandas as pd

excel_file = pd.ExcelFile("../data/Beta Occupational Hazards Dataset_Dec2023.xlsx")
sheet_names = excel_file.sheet_names
sheet_names

['Information', 'BOHD Dataset', 'Variable descriptions', 'Exclusions']

In [7]:
variable_desc_df = pd.read_excel("../data/Beta Occupational Hazards Dataset_Dec2023.xlsx", sheet_name='Variable descriptions', skiprows=3)
variable_desc_df[['Variable', 'Type']].head()

,Variable,Type
0,Electronic Mail,Communication
1,Face-to-Face Discussions,Communication
2,Letters and Memos,Communication
3,Public Speaking,Communication
4,Telephone,Communication


In [9]:
bohd_df = pd.read_excel("../data/Beta Occupational Hazards Dataset_Dec2023.xlsx", sheet_name='BOHD Dataset', skiprows=1)
bohd_df.head()

,Occupation title,ANZSCO code,Electronic Mail,Face-to-Face Discussions,Letters and Memos,Public Speaking,Telephone,Contact With Others,Deal With Physically Aggressive People,Deal With Unpleasant or Angry People,...,Time Pressure,Work Schedules,Degree of Automation,Importance of Being Exact or Accurate,Importance of Repeating Same Tasks,Structured versus Unstructured Work,Employment ('000),Serious claims,Incidence rate,Frequency rate
0,Chief Executives and Managing Directors,1111,100.00,99.50,83.25,59.75,100.00,95.00,21.5,64.25,...,80.00,15.0,42.00,79.00,55.75,99.50,49.490481,370.8,8.359,3.966
1,General Managers,1112,96.25,98.75,73.75,46.75,98.25,94.75,11.0,61.00,...,80.50,7.5,31.75,74.75,64.75,95.00,95.454801,297.8,5.246,2.578
2,Crop Farmers,1212,59.75,95.75,43.75,23.25,78.00,81.75,4.0,30.50,...,62.25,37.0,33.50,62.25,51.50,88.25,35.087945,125.8,5.75,2.599
3,Livestock Farmers,1213,59.75,95.75,43.75,23.25,78.00,81.75,4.0,30.50,...,62.25,37.0,33.50,62.25,51.50,88.25,80.645590,259.2,7.93,3.66
4,Mixed Crop and Livestock Farmers,1214,59.75,95.75,43.75,23.25,78.00,81.75,4.0,30.50,...,62.25,37.0,33.50,62.25,51.50,88.25,25.413240,38,3.686,1.456


### 2. Data Cleaning and Preprocessing

In [10]:
# Checking for missing values
bohd_df.isna().sum()

Occupation title                       0
ANZSCO code                            0
Electronic Mail                        0
Face-to-Face Discussions               0
Letters and Memos                      0
                                      ..
Structured versus Unstructured Work    0
Employment ('000)                      0
Serious claims                         0
Incidence rate                         0
Frequency rate                         0
Length: 63, dtype: int64

In [11]:
bohd_df.dtypes

Occupation title                        object
ANZSCO code                              int64
Electronic Mail                        float64
Face-to-Face Discussions               float64
Letters and Memos                      float64
                                        ...   
Structured versus Unstructured Work    float64
Employment ('000)                      float64
Serious claims                          object
Incidence rate                          object
Frequency rate                          object
Length: 63, dtype: object

In [12]:
# Convert risk scores to numeric format (float64)
bohd_df["Serious claims"] = pd.to_numeric(bohd_df["Serious claims"], errors="coerce")
bohd_df["Incidence rate"] = pd.to_numeric(bohd_df["Incidence rate"], errors="coerce")
bohd_df["Frequency rate"] = pd.to_numeric(bohd_df["Frequency rate"], errors="coerce")

In [13]:
bohd_df[["Serious claims", "Incidence rate", "Frequency rate"]].dtypes

Serious claims    float64
Incidence rate    float64
Frequency rate    float64
dtype: object

In [14]:
# Standardize column names: lowercase, replace spaces with underscores, remove special characters
bohd_df.columns = (
    bohd_df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^a-zA-Z0-9_]", "", regex=True)
)

In [15]:
bohd_df.head()

,occupation_title,anzsco_code,electronic_mail,facetoface_discussions,letters_and_memos,public_speaking,telephone,contact_with_others,deal_with_physically_aggressive_people,deal_with_unpleasant_or_angry_people,...,time_pressure,work_schedules,degree_of_automation,importance_of_being_exact_or_accurate,importance_of_repeating_same_tasks,structured_versus_unstructured_work,employment_000,serious_claims,incidence_rate,frequency_rate
0,Chief Executives and Managing Directors,1111,100.00,99.50,83.25,59.75,100.00,95.00,21.5,64.25,...,80.00,15.0,42.00,79.00,55.75,99.50,49.490481,370.8,8.359,3.966
1,General Managers,1112,96.25,98.75,73.75,46.75,98.25,94.75,11.0,61.00,...,80.50,7.5,31.75,74.75,64.75,95.00,95.454801,297.8,5.246,2.578
2,Crop Farmers,1212,59.75,95.75,43.75,23.25,78.00,81.75,4.0,30.50,...,62.25,37.0,33.50,62.25,51.50,88.25,35.087945,125.8,5.750,2.599
3,Livestock Farmers,1213,59.75,95.75,43.75,23.25,78.00,81.75,4.0,30.50,...,62.25,37.0,33.50,62.25,51.50,88.25,80.645590,259.2,7.930,3.660
4,Mixed Crop and Livestock Farmers,1214,59.75,95.75,43.75,23.25,78.00,81.75,4.0,30.50,...,62.25,37.0,33.50,62.25,51.50,88.25,25.413240,38.0,3.686,1.456
